In [2]:
import requests
import json
import csv
import time

In [3]:
def fetch_item_price(item_id):
    url = base_url + str(item_id)
    response = requests.get(url, headers={
        'User-Agent': '@el on discord'
    })
    response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)
    
    return response.json()    
    
 #   all_data = []
  #  item_data = data.get("data", {}).get(str(item_id), {})

   # if not item_data:
    #    print(f"Item ID {item_id} not found in the API response.")
     #   return None

    #return {
     #   "id": item_id,
      #  "high": item_data.get("avgHighPrice"),
       # "low": item_data.get("avgLowPrice")
    #}

In [4]:
def save_to_csv(data, timestamp, filename="osrs_prices.csv"):
    """
    Appends data to a CSV file.

    Args:
        data (list): List of dictionaries containing item data.
        filename (str): The CSV file to write to.
    """
    file_exists = False
    try:
        # Check if the file already exists
        with open(filename, "r") as file:
            file_exists = True
    except FileNotFoundError:
        pass

    with open(filename, mode="a", newline="") as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(["Item ID", "Timestamp", "Average High Price", "High Price Volume", "Low Price", "Low Price Volume"])  # Header row if file doesn't exist
        for id in data:
            writer.writerow([id, timestamp, data[str(id)]['avgHighPrice'], data[str(id)]['highPriceVolume'], data[str(id)]['avgLowPrice'], data[str(id)]['lowPriceVolume']])
    print(f"Data appended to {filename}")


In [5]:
#Can start at unix time: 1617321600
#86400 to forward the time by one day

In [12]:
base_url = "https://prices.runescape.wiki/api/v1/osrs/24h?timestamp="

print("Old School RuneScape Grand Exchange Price Scraper")

# Example: List of item IDs to fetch
timestamp = 1617321600
enddate = timestamp + (86400*365*2)
count = 0 

while timestamp <= enddate:
    data = fetch_item_price(timestamp)['data']
    save_to_csv(data, timestamp)
    timestamp += 86400
    count += 1
    print('done with', count, 'run')

Old School RuneScape Grand Exchange Price Scraper
Data appended to osrs_prices.csv
done with 1 run
Data appended to osrs_prices.csv
done with 2 run
Data appended to osrs_prices.csv
done with 3 run
Data appended to osrs_prices.csv
done with 4 run
Data appended to osrs_prices.csv
done with 5 run
Data appended to osrs_prices.csv
done with 6 run
Data appended to osrs_prices.csv
done with 7 run
Data appended to osrs_prices.csv
done with 8 run
Data appended to osrs_prices.csv
done with 9 run
Data appended to osrs_prices.csv
done with 10 run
Data appended to osrs_prices.csv
done with 11 run
Data appended to osrs_prices.csv
done with 12 run
Data appended to osrs_prices.csv
done with 13 run
Data appended to osrs_prices.csv
done with 14 run
Data appended to osrs_prices.csv
done with 15 run
Data appended to osrs_prices.csv
done with 16 run
Data appended to osrs_prices.csv
done with 17 run
Data appended to osrs_prices.csv
done with 18 run
Data appended to osrs_prices.csv
done with 19 run
Data appe